# Evaluation

In [ ]:
from collections import defaultdict
import polars as pl

results_checkov = pl.read_json(f'results/checkov_1721775655.json')
results_tflint = pl.read_json(f'results/tflint_1721775655.json')

In [ ]:
CHECKOV_PATTERNS = [
    'Object storage lifecycle rules',
    'Old generation',
    'AWS - Expensive DynamoDB',
]

TFLINT_PATTERNS = [
    'Budget',
    'Object storage lifecycle rules',
    'Old generation',
    'AWS - Expensive DynamoDB',
]

## Precision/recall

### Checkov

#### Before

In [ ]:
checkov_clean = results_checkov.filter([
        pl.col('before').list.len() != 0,
        pl.col('before').list.get(0).struct.field('errors').not_(),
        pl.col('after').struct.field('errors').not_(),
    ])

tp = defaultdict(int)
fp = defaultdict(int)
fn = defaultdict(int)

for real_pattern, url, before, _, _ in checkov_clean.iter_rows():
    matched_patterns = before[0]['matched']

    for pat in CHECKOV_PATTERNS:
        if pat == real_pattern and pat in matched_patterns:
            tp[pat] += 1

        if pat != real_pattern and pat in matched_patterns:
            fp[pat] += 1

            print('----------')
            print('false positive:', url)
            print('commit:', before[0]['id'])
            print('real:', real_pattern)
            print('matched:', matched_patterns)
            for file in before[0]['files']:
                print(file['pattern'], '-', file['lines'], '-', file['path'])

        if pat == real_pattern and pat not in matched_patterns:
            fn[pat] += 1

            print('----------')
            print('false positive:', before[0]['id'])
            print('real:', real_pattern)
            print('matched:', matched_patterns)
            for file in before[0]['files']:
                print(file['pattern'], '-', file['lines'], '-', file['path'])

In [ ]:
pr = []
for pat in CHECKOV_PATTERNS:
    pr.append({
        'pattern': pat,
        'precision': tp[pat] / (tp[pat] + fp[pat]),
        'recall': tp[pat] / (tp[pat] + fn[pat]),
    })

counts_df = checkov_clean.group_by('pattern').len()
measures_df = pl.DataFrame([
    {
        'pattern': p,
        'tp': tp[p],
        'fp': fp[p],
        'fn': fn[p],
    }
    for p in CHECKOV_PATTERNS
])
pr_df = pl.DataFrame(pr)

print(counts_df.join(measures_df, on='pattern').join(pr_df, on='pattern'))

#### Before + after

In [ ]:
tp = defaultdict(int)
fp = defaultdict(int)
fn = defaultdict(int)

for real_pattern, _, before, after, _ in checkov_clean.iter_rows():
    matched_before = before[0]['matched']
    matched_after = after['matched']

    for pat in CHECKOV_PATTERNS:
        if pat == real_pattern and pat in matched_before:
            tp[pat] += 1
        if pat != real_pattern and pat in matched_before:
            fp[pat] += 1
        if pat == real_pattern and pat not in matched_before:
            fn[pat] += 1

        if pat in matched_after:
            fp[pat] += 1

pr = []
for pat in CHECKOV_PATTERNS:
    pr.append({
        'pattern': pat,
        'precision': tp[pat] / (tp[pat] + fp[pat]),
        'recall': tp[pat] / (tp[pat] + fn[pat]),
    })

In [ ]:
pr = []
for pat in CHECKOV_PATTERNS:
    pr.append({
        'pattern': pat,
        'precision': tp[pat] / (tp[pat] + fp[pat]),
        'recall': tp[pat] / (tp[pat] + fn[pat]),
    })

counts_df = checkov_clean.group_by('pattern').len()
measures_df = pl.DataFrame([
    {
        'pattern': p,
        'tp': tp[p],
        'fp': fp[p],
        'fn': fn[p],
    }
    for p in CHECKOV_PATTERNS
])
pr_df = pl.DataFrame(pr)

summary = counts_df.join(measures_df, on='pattern').join(pr_df, on='pattern')

print(summary)

### TFLint

#### Before

In [ ]:
tflint_clean = results_tflint.filter([
        pl.col('before').list.len() != 0,
        pl.col('before').list.get(0).struct.field('errors').not_(),
        pl.col('after').struct.field('errors').not_(),
    ])

tp = defaultdict(int)
fp = defaultdict(int)
fn = defaultdict(int)

for real_pattern, url, before, _, _ in tflint_clean.iter_rows():
    matched_patterns = before[0]['matched']

    for pat in TFLINT_PATTERNS:
        if pat == real_pattern and pat in matched_patterns:
            tp[pat] += 1

        if pat != real_pattern and pat in matched_patterns:
            fp[pat] += 1

            print('----------')
            print('false positive:', url)
            print('commit:', before[0]['id'])
            print('real:', real_pattern)
            print('matched:', matched_patterns)
            for file in before[0]['files']:
                print(file['pattern'], '-', 'start:', file['start'], 'end:', file['end'], '-', file['path'])

        if pat == real_pattern and pat not in matched_patterns:
            fn[pat] += 1

            print('----------')
            print('false negative:', url)
            print('commit:', before[0]['id'])
            print('real:', real_pattern)
            print('matched:', matched_patterns)
            for file in before[0]['files']:
                print(file['pattern'], '-', 'start:', file['start'], 'end:', file['end'], '-', file['path'])

In [ ]:
pr = []
for pat in TFLINT_PATTERNS:
    pr.append({
        'pattern': pat,
        'precision': tp[pat] / (tp[pat] + fp[pat]),
        'recall': tp[pat] / (tp[pat] + fn[pat]),
    })

counts_df = tflint_clean.group_by('pattern').len()
measures_df = pl.DataFrame([
    {
        'pattern': p,
        'tp': tp[p],
        'fp': fp[p],
        'fn': fn[p],
    }
    for p in TFLINT_PATTERNS
])
pr_df = pl.DataFrame(pr)

print(
    counts_df.join(measures_df, on='pattern').join(pr_df, on='pattern')
)

#### Before + after

In [ ]:
tp = defaultdict(int)
fp = defaultdict(int)
fn = defaultdict(int)

for real_pattern, _, before, after, _ in tflint_clean.iter_rows():
    matched_before = before[0]['matched']
    matched_after = after['matched']

    for pat in TFLINT_PATTERNS:
        if pat == real_pattern and pat in matched_before:
            tp[pat] += 1
        if pat != real_pattern and pat in matched_before:
            fp[pat] += 1
        if pat == real_pattern and pat not in matched_before:
            fn[pat] += 1

        if pat in matched_after:
            fp[pat] += 1

pr = []
for pat in TFLINT_PATTERNS:
    pr.append({
        'pattern': pat,
        'precision': tp[pat] / (tp[pat] + fp[pat]),
        'recall': tp[pat] / (tp[pat] + fn[pat]),
    })

In [ ]:
pr = []
for pat in TFLINT_PATTERNS:
    pr.append({
        'pattern': pat,
        'precision': tp[pat] / (tp[pat] + fp[pat]),
        'recall': tp[pat] / (tp[pat] + fn[pat]),
    })

counts_df = tflint_clean.group_by('pattern').len()
measures_df = pl.DataFrame([
    {
        'pattern': p,
        'tp': tp[p],
        'fp': fp[p],
        'fn': fn[p],
    }
    for p in TFLINT_PATTERNS
])
pr_df = pl.DataFrame(pr)

print(
    counts_df.join(measures_df, on='pattern').join(pr_df, on='pattern')
)

### Checkov + TFLint - Latest commits

In [ ]:
latest_matches = defaultdict(lambda: defaultdict(set))

for url, latest in checkov_clean.select('url', 'latest').filter(pl.col('latest').is_not_null()).iter_rows():
    for m in set(latest['matched']):
        prefix, *_ = url.split('/commit/')
        latest_matches[m]['checkov'].add(prefix)

for url, latest in tflint_clean.select('url', 'latest').filter(pl.col('latest').is_not_null()).iter_rows():
    for m in set(latest['matched']):
        prefix, *_ = url.split('/commit/')
        latest_matches[m]['tflint'].add(prefix)

print(pl.DataFrame([
    {
        'pattern': pat,
        'checkov': len(row['checkov']),
        'tflint': len(row['tflint']),
    }
    for pat, row in latest_matches.items()
]))

In [ ]:
from github import Github, Auth
from datetime import datetime, timedelta, UTC

with open('../1-coding/TOKEN.txt', 'r') as token_file:
    token = token_file.readline().strip()
    gh = Github(auth=Auth.Token(token))

print('Using GitHub as', gh.get_user().login)

latest_active_matches = defaultdict(lambda: defaultdict(set))

now = datetime.now(tz=UTC)
threshold = timedelta(days=183)

for pat, row in latest_matches.items():
    for tool, urls in row.items():
        for url in urls:
            try:
                repo = gh.get_repo(url[19:])
                if (now - repo.updated_at) <= threshold or (now - repo.pushed_at) <= threshold:
                    latest_active_matches[pat][tool].add(url)
            except Exception as e:
                print(e)

print(pl.DataFrame([
    {
        'pattern': pat,
        'checkov': len(row['checkov']),
        'tflint': len(row['tflint'])
    }
    for pat, row in latest_active_matches.items()
]))

### Checkov + TFLint - Rule match co-occurrences

In [ ]:
print(
    checkov_clean.select(
        'pattern',
        before=pl.col('before').list.get(0).struct.field('matched').list.len() > 1,
        after=pl.col('after').struct.field('matched').list.len() > 1,
    ).group_by('pattern').sum()
)

print(
    tflint_clean.select(
        'pattern',
        before=pl.col('before').list.get(0).struct.field('matched').list.len() > 1,
        after=pl.col('after').struct.field('matched').list.len() > 1,
    ).group_by('pattern').sum()
)

## Performance

### Summary

In [ ]:
with pl.Config(fmt_str_lengths=500, tbl_width_chars=500):
    print(
        results_checkov
            .select(
                'url',
                duration_before=pl.col('before').list.get(0).struct.field('duration'),
                duration_after=pl.col('after').struct.field('duration'),
            )
            .drop_nulls()
            .top_k(10, by=['duration_before', 'duration_after'])
    )

    print(
        results_tflint
            .select(
                'url',
                duration_before=pl.col('before').list.get(0).struct.field('duration'),
                duration_after=pl.col('after').struct.field('duration'),
            )
            .drop_nulls()
            .top_k(10, by=['duration_before', 'duration_after'])
    )

In [ ]:
checkov_perf = {
    'before': results_checkov.select(duration=pl.col('before').list.get(0).struct.field('duration')).drop_nulls().get_column('duration').to_list(),
    'after': results_checkov.select(duration=pl.col('after').struct.field('duration')).drop_nulls().get_column('duration').to_list(),
    # 'latest': results_checkov.select(duration=pl.col('latest').struct.field('duration')).drop_nulls().get_column('duration').to_list(),
}

tflint_perf = {
    'before': results_tflint.select(duration=pl.col('before').list.get(0).struct.field('duration')).drop_nulls().get_column('duration').to_list(),
    'after': results_tflint.select(duration=pl.col('after').struct.field('duration')).drop_nulls().get_column('duration').to_list(),
    # 'latest': results_tflint.select(duration=pl.col('latest').struct.field('duration')).drop_nulls().get_column('duration').to_list(),
}

In [ ]:
print(
    'Checkov\n* before:\n    mean:',
    statistics.mean(checkov_perf['before']),
    '\n    median:',
    statistics.median(checkov_perf['before']),
    '\n* after:\n    mean:',
    statistics.mean(checkov_perf['after']),
    '\n    median:',
    statistics.median(checkov_perf['after']),
)

print(
    'TFLint\n* before:\n    mean:',
    statistics.mean(tflint_perf['before']),
    '\n    median:',
    statistics.median(tflint_perf['before']),
    '\n* after:\n    mean:',
    statistics.mean(tflint_perf['after']),
    '\n    median:',
    statistics.median(tflint_perf['after']),
)

### Plots

In [ ]:
import statistics
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np

plt.style.use('ggplot')

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 5))

bar_data_checkov = [statistics.mean(v) for v in checkov_perf.values()]
bar_data_tflint = [statistics.mean(v) for v in tflint_perf.values()]

keys = list(checkov_perf.keys())
bar_width = 0.4
x1 = np.arange(len(checkov_perf))
x2 = np.arange(len(tflint_perf)) + len(checkov_perf) + bar_width

bars1 = ax1.bar(x1, bar_data_checkov, bar_width, label='Checkov')
ax1.bar_label(bars1, padding=3)
ax1.set_title('Checkov')
ax1.set_xticks(x1)
ax1.set_xticklabels(keys)
ax1.set_ylabel('Average inspection duration (s)')
ax1.set_yscale('log')
ax1.set_ylim(10**-1, 10**1.7)
ax1.yaxis.set_major_formatter(FuncFormatter(lambda y,pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(y),0)))).format(y)))

bars2 = ax2.bar(x2, bar_data_tflint, bar_width, label='TFLint', color='#348abd')
ax2.bar_label(bars2, padding=3)
ax2.set_title('TFLint')
ax2.set_xticks(x2)
ax2.set_xticklabels(keys)
ax2.set_yscale('log')
ax2.yaxis.set_major_formatter(FuncFormatter(lambda y,pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(y),0)))).format(y)))

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

keys = list(checkov_perf.keys())

ax1.boxplot(list(checkov_perf.values()), showfliers=False)
ax1.set_title('Checkov')
ax1.set_xticklabels(keys)
ax1.set_ylabel('Inspection duration (s)')

ax2.boxplot(list(tflint_perf.values()), showfliers=False)
ax2.set_title('TFLint')
ax2.set_xticklabels(keys)

plt.tight_layout()
plt.show()

In [ ]:
import os

def count_lines(root_dir: str) -> int:
    total = 0
    for base, _, files in os.walk(root_dir):
        for filename in files:
            if filename.endswith(('.tf', '.tf.json')):
                with open(os.path.join(base, filename), 'r') as f:
                    total += sum(1 for _ in f)
    return total

durations_checkov_before = []
lines_checkov_before = []

for url, before in results_checkov.select('url', 'before').iter_rows():
    if len(before) == 0:
        continue

    durations_checkov_before.append(before[0]['duration'])

    _, _, _, owner, repo, _, sha = url.split('/')

    snapshot_path = f'snapshots/{owner}-{repo}-{sha}'

    for version in os.listdir(snapshot_path):
        version_path = f'{snapshot_path}/{version}/'
        if version.startswith('before'):
            lines_checkov_before.append(count_lines(version_path))

durations_tflint_before = []
lines_tflint_before = []

for url, before in results_tflint.select('url', 'before').iter_rows():
    if len(before) == 0:
        continue

    durations_tflint_before.append(before[0]['duration'])

    _, _, _, owner, repo, _, sha = url.split('/')

    snapshot_path = f'snapshots/{owner}-{repo}-{sha}'

    for version in os.listdir(snapshot_path):
        version_path = f'{snapshot_path}/{version}/'
        if version.startswith('before'):
            lines_tflint_before.append(count_lines(version_path))

assert len(durations_checkov_before) == len(lines_checkov_before)
assert len(durations_tflint_before) == len(lines_tflint_before)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

ax1.scatter(y=durations_checkov_before, x=lines_checkov_before)
ax1.set_title('Checkov')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylabel('Total inspection duration (s)')

ax2.scatter(y=durations_tflint_before, x=lines_tflint_before, color='#348abd')
ax2.set_title('TFLint')
ax2.set_xscale('log')
ax2.set_yscale('log')

fig.supxlabel('Total lines of IaC code', color=ax1.yaxis.label.get_color())

plt.show()